In [11]:
from dataloader import *
from model import *
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm_notebook
from torch.utils.data import DataLoader
from loss import *

### Let us Load the data

In [5]:
dataset_args = {
    "path": './data/',
    "patchsize": (64,64),
    "margin":(80,80)
}

In [6]:
dataloader_args = {
    "batch_size":16,
    "num_workers":1
}

In [10]:
traindata = cevae(**dataset_args)
train_loader = DataLoader(traindata,**dataloader_args)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(epoch, model, optimizer, train_loader, device, tx, log_var_std, lamda=0.5, beta=1.0):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = inpt,inpt_noisy
        inpt = inpt.to(device)
        inpt_noisy = inpt_noisy.to(device)

        optimizer.zero_grad()
        x_rec_vae, z_dist, = model(inpt)
        x_rec_ce = model(inpt_noisy)

        # kl_loss_low = kl_loss(z_dist_low)
        kl_loss = kl_loss_fn(z_dist)
        rec_loss_vae = rec_loss_fn(x_rec_vae, inpt)
        loss_vae = kl_loss * beta + rec_loss_vae

        rec_loss_ce = rec_loss_fn(x_rec_ce, inpt)

        loss_ce = rec_loss_ce

        loss = (1. - lamda) * loss_vae + lamda * loss_ce

        loss.backward()
        train_loss += loss.item()
        optimizer.step()
